# LLM as a judge for Transcripts (Doctor and Patient Chatbots) - version 2.0

Some improvements to tackle:
- give some reasoning as to why the model is marking things as bad

In [1]:
import json
from pprint import pprint

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from dotenv import load_dotenv

# Load API keys from .env file
load_dotenv()

True

In [21]:
# Create the ChatOpenAI model instance

# model_name = "gpt-3.5-turbo-0125"  # release date: 2024-01-25
# model_name = "gpt-3.5-turbo-1106"  # release date: 2023-11-06
model_name = "gpt-4o-2024-05-13"  # release date: 2024-05-13

model = ChatOpenAI(temperature=0.0, model_name=model_name)

### Specify patient transcript file to read in
transcripts_version = "1.0"

## System Prompt

In [6]:
# Define the system message for the evaluation

symptom_ask_description = "Did the DOCTOR ask about {symptom}?"

judge_criteria = {
    # introduction
    "patient_name": "Was the PATIENT's name mentioned by the DOCTOR?",

    # symptoms
    "dyspnea": symptom_ask_description.format(symptom="dyspnea (shortness of breath) at rest, while walking or climbing stairs"),
    "pnd": symptom_ask_description.format(symptom="paroxysmal nocturnal dyspnea (PND) (sudden shortness of breath at night)"),
    "orthopnea": symptom_ask_description.format(symptom="orthopnea (shortness of breath while lying flat)"),
    "ankle_edema": symptom_ask_description.format(symptom="ankle edema or general lower extremity swelling (swelling in ankles or legs)"),
    "nocturnal_cough": symptom_ask_description.format(symptom="nocturnal cough (coughing at night)"),
    "chest_pain": symptom_ask_description.format(symptom="chest pain"),
    "fatigue": symptom_ask_description.format(symptom="fatigue"),
    "worsening_mental_status": symptom_ask_description.format(symptom="worsening mental status that is acute (sudden confusion or mental changes)"),

    # medications
    "doctor_ask_medications": "Did the DOCTOR ask the PATIENT what medications they are on?",

    # vitals
    "temperature": "Did the DOCTOR ask for the PATIENT's temperature/body temperature?",
    "heart_rate": "Did the DOCTOR ask for the PATIENT's heart rate (pulse)?",
    "respiratory_rate": "Did the DOCTOR ask for the PATIENT's respiratory rate (number of breaths per minute)?",
    "oxygen_saturation": "Did the DOCTOR ask for the PATIENT's oxygen saturation (oxygen level in the blood)?",
    "blood_pressure": "Did the DOCTOR ask for the PATIENT's blood pressure (systolic and diastolic / upper and lower blood pressure numbers)?",
    "weight": "Did the DOCTOR ask for the PATIENT's weight (body weight)?",

    # hcp_quality
    "sympathetic_patient": "Was the DOCTOR sympathetic when the PATIENT reported symptoms or pain?",
    "reminder": "Did the DOCTOR remind the PATIENT to contact their healthcare provider if they notice any significant changes or worsening of symptoms?",
    "end_conversation": "Did the DOCTOR express care and encourage the PATIENT to reach out if they need further assistance at the end of the conversation?",
    "natural_conversation": "Did the conversation flow naturally without repetition?",
    "no_premature_end": "The conversation ended properly without a premature end.",

    # patient_quality
    "plain_language": "Did the PATIENT use plain language?",
    "consistent_symptoms": "Was the PATIENT consistent about their symptoms?",
    "no_confabulations": "Did the PATIENT avoid inventing information that contradicts the prompt (confabulations)?",
    "allow_doctor_questions": "Did the PATIENT allow the DOCTOR to ask questions and not take over the direction of the conversation (offering information before it’s asked for)?",
}

system_message_summary_judge = """You are evaluating a dialogue TRANSCRIPT generated by a DOCTOR chatbot and a PATIENT LLM bot, based on a set of criteria.
Your evaluation will consist of answering specific questions about the DOCTOR/PATIENT bot with 1 (Yes) and 0 (No) responses.
The DOCTOR bot quality should not depend on the PATIENT.
The PATIENT bot quality however does depend on its own prompt (PATIENT_PROMPT).
{output_format}

CRITERIA (column name, then a description):
""" + "\n".join([f"{k}, {v}" for k, v in judge_criteria.items()])

output_csv_format = """Generate a CSV row with the appropriate 1 or 0 for each criteria in the order specified below."""

output_reasoning_format = """In separate lines, state each criteria's value (1 or 0) and briefly explain your reasoning if it's a 0. When explaining reasoning, be very specific and please refer to texts in the TRANSCRIPT that is the offender. If it's a 1 (yes), leave the reasoning empty.
Lastly, in one last new line, please provide any short additional observations or suggestions for improvement (1 sentence), but do not repeat evaluation points previously made.
For example:
patient_name,1,""
dyspnea,1,""
pnd,0,"The DOCTOR did not ask about PND in the conversation."
sympathetic_patient,0,"The DOCTOR ignored after PATIENT writes 'I am feeling light-headed.'"
consistent_symptoms,0,"The PATIENT says 'I have chest pain' but later says 'I have no chest pain.'"
write your one-sentence observation/improvement here
"""

human_message_summary_judge = """
PATIENT_PROMPT: {patient_prompt}

TRANSCRIPT: {transcript}
"""

In [7]:
pprint(system_message_summary_judge.format(output_format=output_reasoning_format))

('You are evaluating a dialogue TRANSCRIPT generated by a DOCTOR chatbot and a '
 'PATIENT LLM bot, based on a set of criteria.\n'
 'Your evaluation will consist of answering specific questions about the '
 'DOCTOR/PATIENT bot with 1 (Yes) and 0 (No) responses.\n'
 'The DOCTOR bot quality should not depend on the PATIENT.\n'
 'The PATIENT bot quality however does depend on its own prompt '
 '(PATIENT_PROMPT).\n'
 "In separate lines, state each criteria's value (1 or 0) and briefly explain "
 "your reasoning if it's a 0. When explaining reasoning, be very specific and "
 "please refer to texts in the TRANSCRIPT that is the offender. If it's a 1 "
 '(yes), leave the reasoning empty.\n'
 'Lastly, in one last new line, please provide any short additional '
 'observations or suggestions for improvement (1 sentence), but do not repeat '
 'evaluation points previously made.\n'
 'For example:\n'
 'patient_name,1,""\n'
 'dyspnea,1,""\n'
 'pnd,0,"The DOCTOR did not ask about PND in the conversat

## Import Transcript

In [26]:
# Specify the path to your JSON file
transcripts_json_file_path = f"../../data/patients/patients_{transcripts_version}_with_transcripts.json"

# Open and read the JSON file
with open(transcripts_json_file_path, 'r') as json_file:
    transcripts = json.load(json_file)

# Specify the CSV file path (make sure it is a file, not a directory)
csv_file_path = f"../../data/evaluations/transcripts_{transcripts_version}_evaluation_2.csv"

# # -----
# # for testing only; comment when not needed -- only try to generate one summary
# sample_patient_idx = 3
# transcripts = {k: v for k, v in transcripts.items() if k == list(transcripts.keys())[sample_patient_idx]}
# sole_patient_id = list(transcripts.keys())[0]
# # -----

In [10]:
# Function to validate and parse the response

# Example response
#  'intro_patient,1,""\n'
#  'current_symptoms,1,""\n'
#  'symptoms_agree,0,"Nose bleeding was mentioned in the summary, but not in the transcript."\n'

# Desired output
# {"intro_patient": {"value": 1, "reasoning": ""}, "current_symptoms": {"value": 1, "reasoning": ""}, ...}

def parse_response(response_content: str, expected_fields=len(judge_criteria)):
    response_list = response_content.split("\n")[0:expected_fields]
    if len(response_list) != expected_fields:
        return {"error": "Invalid response count"}
    response_dict = {}
    for response in response_list:
        if response:
            # split by first two commas, but keep the rest of the string
            response_split = response.split(",", 2)
            response_dict[response_split[0]] = {"value": int(response_split[1]), "reasoning": response_split[2].strip('"')}

    # remainder text is the observations
    response_dict["observations"] = "\n".join(response_content.split("\n")[expected_fields:]).strip()

    return response_dict

In [27]:
# Write the header to dataframe
# for each criteria, add two columns: one for the value (same name) and one for the reasoning (suffix _reasoning)
column_order = ["transcript_number"]
for criteria in judge_criteria.keys():
    column_order.append(criteria)
    column_order.append(f"{criteria}_reasoning")
column_order.append("observations")

all_rows_series: list[pd.Series] = []
all_responses = []
# Loop through each transcript number, invoke the model, and write the results
for patient_number in transcripts.keys():
    if patient_number in transcripts:
        patient_data = transcripts[patient_number]
        transcript = patient_data['chat_transcript']
        patient_prompt = patient_data['prompt']

        prompt = (
            SystemMessage(content=system_message_summary_judge.format(output_format=output_reasoning_format))
            + human_message_summary_judge
        )

        # Get the response
        response = model.invoke(
            prompt.format_messages(transcript=transcript, patient_prompt=patient_prompt)
        )
        all_responses.append(response)
        response_dict = parse_response(response.content)

        # add to dataframe
        row_to_add = {
            "transcript_number": patient_number,
            **{k: v["value"] for k, v in response_dict.items() if k in judge_criteria.keys()},
            **{f"{k}_reasoning": v["reasoning"] for k, v in response_dict.items() if k in judge_criteria.keys()},
            "observations": response_dict["observations"]
        }
        all_rows_series.append(pd.Series(row_to_add))

# create dataframe
df = pd.DataFrame(all_rows_series)[column_order]

display(df)

# Write the dataframe to a CSV file
df.to_csv(csv_file_path, index=False)
print(f"CSV file has been created at: {csv_file_path}")

,transcript_number,patient_name,patient_name_reasoning,dyspnea,dyspnea_reasoning,pnd,pnd_reasoning,orthopnea,orthopnea_reasoning,ankle_edema,ankle_edema_reasoning,nocturnal_cough,nocturnal_cough_reasoning,chest_pain,chest_pain_reasoning,fatigue,fatigue_reasoning,worsening_mental_status,worsening_mental_status_reasoning,doctor_ask_medications,doctor_ask_medications_reasoning,temperature,temperature_reasoning,heart_rate,heart_rate_reasoning,respiratory_rate,respiratory_rate_reasoning,oxygen_saturation,oxygen_saturation_reasoning,blood_pressure,blood_pressure_reasoning,weight,weight_reasoning,sympathetic_patient,sympathetic_patient_reasoning,reminder,reminder_reasoning,end_conversation,end_conversation_reasoning,natural_conversation,natural_conversation_reasoning,no_premature_end,no_premature_end_reasoning,plain_language,plain_language_reasoning,consistent_symptoms,consistent_symptoms_reasoning,no_confabulations,no_confabulations_reasoning,allow_doctor_questions,allow_doctor_questions_reasoning,observations
0,12305811,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,The DOCTOR did not ask for the PATIENT's weight.,1,,0,The DOCTOR did not remind the PATIENT to contact their healthcare provider if symptoms worsen.,0,The DOCTOR did not express care or encourage the PATIENT to reach out if they need further assistance at the end of the conversation.,0,"The conversation repeated itself, starting over with 'Hello, Kevin. I'm here to check on how you're feeling today.'",0,"The conversation ended abruptly with a repeated question, without a proper conclusion.",1,,1,,1,,1,,The DOCTOR should avoid repeating the same questions and ensure a proper conclusion to the conversation.
1,14185111,1,,1,,1,,1,,1,,0,The DOCTOR did not ask about nocturnal cough in the conversation.,1,,1,,0,The DOCTOR did not ask about worsening mental status that is acute.,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,"The conversation was repeated almost verbatim in the second half, indicating a lack of natural flow.",1,,1,,1,,1,,1,,The conversation should avoid repetition to maintain a natural flow and ensure all relevant symptoms are covered.
2,10339317,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,The DOCTOR did not ask for the PATIENT's blood pressure.,1,,1,,0,The DOCTOR did not remind the PATIENT to contact their healthcare provider if they notice any significant changes or worsening of symptoms.,0,The DOCTOR did not express care or encourage the PATIENT to reach out if they need further assistance at the end of the conversation.,0,"The conversation was repetitive, with the DOCTOR asking the same questions multiple times.",1,,1,,1,,1,,1,,The DOCTOR should avoid repetitive questioning and ensure to ask for the patient's blood pressure.
3,14807966,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,The DOCTOR did not remind the PATIENT to contact their healthcare provider if they notice any significant changes or worsening of symptoms.,0,The DOCTOR did not express care or encourage the PATIENT to reach out if they need further assistance at the end of the conversation.,1,,1,,1,,1,,1,,1,,The DOCTOR should include a reminder for the PATIENT to contact their healthcare provider if symptoms worsen and express care at the end of the conversation.
4,13912736,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,"The conversation repeated itself starting from 'Hello, I'm here to check on how you're feeling today.'",1,,1,,1,,1,,1,,The conversation should avoid repetition to maintain a natural flow.
5,15338322,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,"The conversation repeated questions and responses, making it feel unnatural (e.g., 'Have you had any sudden shortness of breath that wakes you up at night?' was asked twice).",1,,1,,1,,1,,1,,The DOCTOR should avoid repeating questions to maintain a more natural conversation flow.
6,13166275,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,1,,0,The DOCTOR did not ask for the PATIENT's weight.,1,,0,The DOCTOR did not remind the P

CSV file has been created at: ../../data/evaluations/transcripts_1.0_evaluation_2.csv


In [23]:
parse_response(response.content)

{'patient_name': {'value': 1, 'reasoning': ''},
 'dyspnea': {'value': 1, 'reasoning': ''},
 'pnd': {'value': 1, 'reasoning': ''},
 'orthopnea': {'value': 1, 'reasoning': ''},
 'ankle_edema': {'value': 1, 'reasoning': ''},
 'nocturnal_cough': {'value': 1, 'reasoning': ''},
 'chest_pain': {'value': 1, 'reasoning': ''},
 'fatigue': {'value': 1, 'reasoning': ''},
 'worsening_mental_status': {'value': 1, 'reasoning': ''},
 'doctor_ask_medications': {'value': 1, 'reasoning': ''},
 'temperature': {'value': 1, 'reasoning': ''},
 'heart_rate': {'value': 1, 'reasoning': ''},
 'respiratory_rate': {'value': 1, 'reasoning': ''},
 'oxygen_saturation': {'value': 1, 'reasoning': ''},
 'blood_pressure': {'value': 1, 'reasoning': ''},
 'weight': {'value': 1, 'reasoning': ''},
 'sympathetic_patient': {'value': 1, 'reasoning': ''},
 'reminder': {'value': 0,
  'reasoning': 'The DOCTOR did not remind the PATIENT to contact their healthcare provider if they notice any significant changes or worsening of symp

In [38]:
# what's an average response resource usage?
print(response.response_metadata)

# what's the total resource usage?
prompt_cost = 5 / 1e6  # $5 per 1M tokens
completion_cost = 15 / 1e6  # $15 per 1M tokens
total_prompt_tokens = sum([r.response_metadata["token_usage"]["prompt_tokens"] for r in all_responses])
total_completion_tokens = sum([r.response_metadata["token_usage"]["completion_tokens"] for r in all_responses])
print(f"Total prompt tokens: {total_prompt_tokens}, cost: ${(total_prompt_cost := total_prompt_tokens * prompt_cost):0.3f}, average: {total_prompt_tokens / len(all_responses)} tokens per response")
print(f"Total completion tokens: {total_completion_tokens}, cost: ${(total_completion_cost := total_completion_tokens * completion_cost):0.3f}, average: {total_completion_tokens / len(all_responses)} tokens per response")
print(f"Total cost: ${total_prompt_cost + total_completion_cost:0.3f}")

{'token_usage': {'completion_tokens': 238, 'prompt_tokens': 2647, 'total_tokens': 2885}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'stop', 'logprobs': None}
Total prompt tokens: 54653, cost: $0.273, average: 2732.65 tokens per response
Total completion tokens: 4940, cost: $0.074, average: 247.0 tokens per response
Total cost: $0.347


In [ ]:
from pprint import pprint

patient_id = "13727153"
pprint(transcripts[patient_id]["chat_transcript"])
pprint(summaries[patient_id]["summary"])

In [39]:
# viewer: open the csv file as a pandas dataframe

import pandas as pd

# view options -- max number of columns and column width
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)

df = pd.read_csv(csv_file_path)

display(df)

,transcript_number,patient_name,patient_name_reasoning,dyspnea,dyspnea_reasoning,pnd,pnd_reasoning,orthopnea,orthopnea_reasoning,ankle_edema,ankle_edema_reasoning,nocturnal_cough,nocturnal_cough_reasoning,chest_pain,chest_pain_reasoning,fatigue,fatigue_reasoning,worsening_mental_status,worsening_mental_status_reasoning,doctor_ask_medications,doctor_ask_medications_reasoning,temperature,temperature_reasoning,heart_rate,heart_rate_reasoning,respiratory_rate,respiratory_rate_reasoning,oxygen_saturation,oxygen_saturation_reasoning,blood_pressure,blood_pressure_reasoning,weight,weight_reasoning,sympathetic_patient,sympathetic_patient_reasoning,reminder,reminder_reasoning,end_conversation,end_conversation_reasoning,natural_conversation,natural_conversation_reasoning,no_premature_end,no_premature_end_reasoning,plain_language,plain_language_reasoning,consistent_symptoms,consistent_symptoms_reasoning,no_confabulations,no_confabulations_reasoning,allow_doctor_questions,allow_doctor_questions_reasoning,observations
0,12305811,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,The DOCTOR did not ask for the PATIENT's weight.,1,NaN,0,The DOCTOR did not remind the PATIENT to contact their healthcare provider if symptoms worsen.,0,The DOCTOR did not express care or encourage the PATIENT to reach out if they need further assistance at the end of the conversation.,0,"The conversation repeated itself, starting over with 'Hello, Kevin. I'm here to check on how you're feeling today.'",0,"The conversation ended abruptly with a repeated question, without a proper conclusion.",1,NaN,1,NaN,1,NaN,1,NaN,The DOCTOR should avoid repeating the same questions and ensure a proper conclusion to the conversation.
1,14185111,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,The DOCTOR did not ask about nocturnal cough in the conversation.,1,NaN,1,NaN,0,The DOCTOR did not ask about worsening mental status that is acute.,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,"The conversation was repeated almost verbatim in the second half, indicating a lack of natural flow.",1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,The conversation should avoid repetition to maintain a natural flow and ensure all relevant symptoms are covered.
2,10339317,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,The DOCTOR did not ask for the PATIENT's blood pressure.,1,NaN,1,NaN,0,The DOCTOR did not remind the PATIENT to contact their healthcare provider if they notice any significant changes or worsening of symptoms.,0,The DOCTOR did not express care or encourage the PATIENT to reach out if they need further assistance at the end of the conversation.,0,"The conversation was repetitive, with the DOCTOR asking the same questions multiple times.",1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,The DOCTOR should avoid repetitive questioning and ensure to ask for the patient's blood pressure.
3,14807966,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,The DOCTOR did not remind the PATIENT to contact their healthcare provider if they notice any significant changes or worsening of symptoms.,0,The DOCTOR did not express care or encourage the PATIENT to reach out if they need further assistance at the end of the conversation.,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,The DOCTOR should include a reminder for the PATIENT to contact their healthcare provider if symptoms worsen and express care at the end of the conversation.
4,13912736,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,"The conversation repeated itself starting from 'Hello, I'm here to check on how you're feeling today.'",1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,The conversation should avoid repetition to maintain a natural flow.
5,15338322,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,"The conversation repeated questions a